In [1]:
# %matplotlib notebook
%matplotlib inline

%load_ext rpy2.ipython

# %connect_info

import jzap.misc
text = """
Jupyter connect hash:
{word}

To get matching file:
    $> python -c "import jzap.misc; print(jzap.misc.notebook_file('{word}', '/home/jeff/.local/share/jupyter/runtime'))"
To connect to existing notebook:
    $> jupyter console --existing FILE
    
"""
print(text.format(word=jzap.misc.notebook_word()))
print("Last run: {}".format(jzap.misc.timestamp()))

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi


Jupyter connect hash:
forge

To get matching file:
    $> python -c "import jzap.misc; print(jzap.misc.notebook_file('forge', '/home/jeff/.local/share/jupyter/runtime'))"
To connect to existing notebook:
    $> jupyter console --existing FILE
    

Last run: 2018-09-06-13h46m57s


In [2]:
import numpy as np
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols, mixedlm
from statsmodels.stats.anova import anova_lm
import patsy
import matplotlib.pyplot as plt
import seaborn as sns

import jzap
import replay.lib.settings

import warnings
warnings.filterwarnings('ignore')

path = '/lab/data/behavior_replay_180905.txt'

data = jzap.misc.load_table(path)
print(data.columns)
print(data.head())

/usr/local/lib/python2.7/dist-packages/statsmodels/tsa/statespace/tools.py:12: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _statespace
/usr/local/lib/python2.7/dist-packages/statsmodels/tsa/statespace/tools.py:12: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from . import _statespace
/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python2.7/dist-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:33: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import kalman_loglike
/usr/local/lib/python2.7/dist-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:33: RuntimeWarning: numpy.ufunc size 

Index([u'mouse', u'date', u'temporal_prior_events_0_5', u'dprime',
       u'hmm_dprime', u'hmm_dprime_noneutral', u'hmm_dprime_run2',
       u'hmm_dprime_run3', u'hmm_dprime_run4', u'hmm_engagement',
       ...
       u'replay_freq_0_1_all', u'replay_freq_0_1_all_prev',
       u'combrep_freq_0_1_all', u'combrep_freq_0_1_all_prev',
       u'replay_freq_hungry_0_1_all', u'replay_freq_hungry_0_1_all_prev',
       u'replay_freq_iti_0_1_all', u'replay_freq_iti_0_1_all_prev',
       u'd_day_next_3', u'd_day_prev_3'],
      dtype='object', length=133)
  mouse       date  temporal_prior_events_0_5  dprime  hmm_dprime  \
0  AS20 2016-08-16                     0.0189   3.278       3.288   
1  AS20 2016-08-18                     0.0302   3.863       3.873   
2  AS20 2016-08-19                     0.0285   1.931       2.581   
3  AS20 2016-08-22                     0.0219   3.696       3.709   
4  AS20 2016-08-23                     0.0241   3.961       2.282   

   hmm_dprime_noneutral  hmm_dprim

In [3]:
formula = 'd_hmm_dprime_next ~ replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral + d_day_next'
data_sub = data[[
    'd_hmm_dprime_next', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_next']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

                    Results: Generalized linear model
Model:                 GLM                  AIC:               206.3347 
Link Function:         identity             BIC:               -179.8866
Dependent Variable:    d_hmm_dprime_next    Log-Likelihood:    -98.167  
Date:                  2018-09-06 13:46     LL-Null:           -98.493  
No. Observations:      66                   Deviance:          75.682   
Df Model:              4                    Pearson chi2:      75.7     
Df Residuals:          61                   Scale:             1.2407   
Method:                IRLS                                             
------------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025   0.975]
------------------------------------------------------------------------
Intercept                0.2005   0.2255  0.8888 0.3741  -0.2416  0.6425
replay_freq_0_1_plus     5.3496  34.0072  0.1573 0.8750 -61.3034 72.00

In [4]:
formula = 'd_hmm_dprime_next ~ 1'
data_sub = data[[
    'd_hmm_dprime_next', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_next']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

               Results: Generalized linear model
Model:              GLM               AIC:            198.9853 
Link Function:      identity          BIC:            -195.8954
Dependent Variable: d_hmm_dprime_next Log-Likelihood: -98.493  
Date:               2018-09-06 13:46  LL-Null:        -98.493  
No. Observations:   66                Deviance:       76.432   
Df Model:           0                 Pearson chi2:   76.4     
Df Residuals:       65                Scale:          1.1759   
Method:             IRLS                                       
----------------------------------------------------------------
               Coef.   Std.Err.    z     P>|z|    [0.025  0.975]
----------------------------------------------------------------
Intercept      0.1692    0.1335  1.2675  0.2050  -0.0924  0.4308



In [5]:
formula = 'd_hmm_dprime_prev ~ replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral + d_day_prev'
data_sub = data[[
    'd_hmm_dprime_prev', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_prev']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

                      Results: Generalized linear model
Model:                   GLM                   AIC:                225.5006 
Link Function:           identity              BIC:                -187.1021
Dependent Variable:      d_hmm_dprime_prev     Log-Likelihood:     -107.75  
Date:                    2018-09-06 13:46      LL-Null:            -110.45  
No. Observations:        70                    Deviance:           89.050   
Df Model:                4                     Pearson chi2:       89.1     
Df Residuals:            65                    Scale:              1.3700   
Method:                  IRLS                                               
----------------------------------------------------------------------------
                          Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
----------------------------------------------------------------------------
Intercept                  0.4742   0.2245  2.1126 0.0346    0.0343   0.9142
replay_freq_0_1_plus

In [6]:
AIC_min = 206.3448
AIC_other = 225.5649

print(np.exp((AIC_min-AIC_other)/2.))

6.705147164541301e-05


In [7]:
formula = 'hmm_dprime_next ~ replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral + hmm_dprime'
data_sub = data[[
    'hmm_dprime_next', 'hmm_dprime', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_next']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

                     Results: Generalized linear model
Model:                  GLM                  AIC:                187.1107 
Link Function:          identity             BIC:                -199.0105
Dependent Variable:     hmm_dprime_next      Log-Likelihood:     -88.555  
Date:                   2018-09-06 13:46     LL-Null:            -102.60  
No. Observations:       66                   Deviance:           56.558   
Df Model:               4                    Pearson chi2:       56.6     
Df Residuals:           61                   Scale:              0.92719  
Method:                 IRLS                                              
--------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
--------------------------------------------------------------------------
Intercept                 1.3909   0.3231  4.3041 0.0000    0.7575  2.0242
replay_freq_0_1_plus    -12.5046  29.6214 -0.

In [8]:
formula = 'hmm_dprime_next ~ hmm_dprime'
data_sub = data[[
    'hmm_dprime_next', 'hmm_dprime', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_next']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

               Results: Generalized linear model
Model:              GLM              AIC:            183.7837 
Link Function:      identity         BIC:            -209.2419
Dependent Variable: hmm_dprime_next  Log-Likelihood: -89.892  
Date:               2018-09-06 13:46 LL-Null:        -102.60  
No. Observations:   66               Deviance:       58.896   
Df Model:           1                Pearson chi2:   58.9     
Df Residuals:       64               Scale:          0.92025  
Method:             IRLS                                      
---------------------------------------------------------------
               Coef.   Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------------------
Intercept      1.0333    0.2305  4.4829  0.0000  0.5815  1.4850
hmm_dprime     0.5568    0.1015  5.4841  0.0000  0.3578  0.7558



In [9]:
formula = 'hmm_dprime ~ replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral + hmm_dprime_prev'
data_sub = data[[
    'hmm_dprime_prev', 'hmm_dprime', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_prev']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

                     Results: Generalized linear model
Model:                  GLM                  AIC:                206.0910 
Link Function:          identity             BIC:                -208.6663
Dependent Variable:     hmm_dprime           Log-Likelihood:     -98.045  
Date:                   2018-09-06 13:46     LL-Null:            -113.35  
No. Observations:       70                   Deviance:           67.486   
Df Model:               4                    Pearson chi2:       67.5     
Df Residuals:           65                   Scale:              1.0382   
Method:                 IRLS                                              
--------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
--------------------------------------------------------------------------
Intercept                 1.5603   0.3041  5.1312 0.0000    0.9643  2.1563
replay_freq_0_1_plus    -33.6746  24.5438 -1.

In [10]:
formula = 'hmm_dprime ~ hmm_dprime_prev'
data_sub = data[[
    'hmm_dprime_prev', 'hmm_dprime', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_prev']]

# 0s in the dependent variable still converges, but LL is infinite
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)

# family = sm.families.Gamma()
# family = sm.families.Gamma(link=sm.families.links.log)
family = sm.families.Gaussian()
# family = sm.families.Gaussian(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())

               Results: Generalized linear model
Model:              GLM              AIC:            208.5416 
Link Function:      identity         BIC:            -212.7525
Dependent Variable: hmm_dprime       Log-Likelihood: -102.27  
Date:               2018-09-06 13:46 LL-Null:        -113.35  
No. Observations:   70               Deviance:       76.145   
Df Model:           1                Pearson chi2:   76.1     
Df Residuals:       68               Scale:          1.1198   
Method:             IRLS                                      
--------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
--------------------------------------------------------------
Intercept          1.0563   0.2451 4.3092 0.0000 0.5758 1.5367
hmm_dprime_prev    0.5432   0.1080 5.0312 0.0000 0.3316 0.7548



## Model Behavior

In [11]:
# data_sub = data[[
#     'hmm_dprime_prev', 'hmm_dprime', 'hmm_dprime_next', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
#     'replay_freq_0_1_neutral', 'd_day_prev', 'd_day_next', 'replay_plus_prev', 'replay_minus_prev',
#     'replay_neutral_prev', 'replay_plus_next', 'replay_minus_next', 'replay_neutral_next']]
data_sub = data[[
    'hmm_dprime_prev', 'hmm_dprime', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
    'replay_freq_0_1_neutral', 'd_day_prev', 'replay_plus_prev', 'replay_minus_prev',
    'replay_neutral_prev']]
# data_sub = data[[
#     'd_hmm_dprime_prev', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
#     'replay_freq_0_1_neutral', 'd_day_prev', 'replay_plus_prev', 'replay_minus_prev',
#     'replay_neutral_prev']]
# data_sub.replace(0, np.nan, inplace=True)
print(data_sub.shape)
data_sub.dropna(inplace=True)
print(data_sub.shape)

(118, 9)
(82, 9)


In [12]:
# Full
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev ' \
    + '+ replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral ' \
    + '+ replay_plus_prev + replay_minus_prev + replay_neutral_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_full = glm_results.aic

                     Results: Generalized linear model
Model:                  GLM                  AIC:                241.5002 
Link Function:          identity             BIC:                -248.3983
Dependent Variable:     hmm_dprime           Log-Likelihood:     -111.75  
Date:                   2018-09-06 13:46     LL-Null:            -127.57  
No. Observations:       82                   Deviance:           73.292   
Df Model:               8                    Pearson chi2:       73.3     
Df Residuals:           73                   Scale:              1.0040   
Method:                 IRLS                                              
--------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
--------------------------------------------------------------------------
Intercept                 1.4456   0.3272  4.4183 0.0000    0.8044  2.0869
hmm_dprime_prev           0.4782   0.1146  4.

In [13]:
# Null
formula = 'hmm_dprime ~ 1'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_null = glm_results.aic

               Results: Generalized linear model
Model:              GLM              AIC:            257.1359 
Link Function:      identity         BIC:            -249.1466
Dependent Variable: hmm_dprime       Log-Likelihood: -127.57  
Date:               2018-09-06 13:46 LL-Null:        -127.57  
No. Observations:   82               Deviance:       107.80   
Df Model:           0                Pearson chi2:   108.     
Df Residuals:       81               Scale:          1.3308   
Method:             IRLS                                      
---------------------------------------------------------------
              Coef.   Std.Err.     z     P>|z|   [0.025  0.975]
---------------------------------------------------------------
Intercept     2.2238    0.1274  17.4561  0.0000  1.9741  2.4735



In [14]:
# Autoregress
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_ar = glm_results.aic

               Results: Generalized linear model
Model:              GLM              AIC:            234.8354 
Link Function:      identity         BIC:            -269.9115
Dependent Variable: hmm_dprime       Log-Likelihood: -114.42  
Date:               2018-09-06 13:46 LL-Null:        -127.57  
No. Observations:   82               Deviance:       78.219   
Df Model:           2                Pearson chi2:   78.2     
Df Residuals:       79               Scale:          0.99012  
Method:             IRLS                                      
--------------------------------------------------------------
                 Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
--------------------------------------------------------------
Intercept        1.1362   0.2462  4.6148 0.0000  0.6536 1.6187
hmm_dprime_prev  0.5477   0.1005  5.4509 0.0000  0.3508 0.7446
d_day_prev      -0.0155   0.0283 -0.5493 0.5828 -0.0710 0.0399



In [15]:
# Arthur 1
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev + replay_plus_prev + replay_minus_prev + replay_neutral_prev'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_plus_prev + replay_minus_prev + replay_neutral_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a1 = glm_results.aic

                   Results: Generalized linear model
Model:                  GLM                AIC:              239.5813 
Link Function:          identity           BIC:              -257.8784
Dependent Variable:     hmm_dprime         Log-Likelihood:   -113.79  
Date:                   2018-09-06 13:46   LL-Null:          -127.57  
No. Observations:       82                 Deviance:         77.032   
Df Model:               5                  Pearson chi2:     77.0     
Df Residuals:           76                 Scale:            1.0136   
Method:                 IRLS                                          
----------------------------------------------------------------------
                     Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
----------------------------------------------------------------------
Intercept             1.3236   0.3085  4.2904 0.0000    0.7190  1.9283
hmm_dprime_prev       0.5011   0.1109  4.5189 0.0000    0.2838  0.7184
d_day_prev           -0.

In [16]:
# Arthur 2
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a2 = glm_results.aic

                    Results: Generalized linear model
Model:                   GLM                 AIC:               235.7266 
Link Function:           identity            BIC:               -261.4158
Dependent Variable:      hmm_dprime          Log-Likelihood:    -111.86  
Date:                    2018-09-06 13:46    LL-Null:           -127.57  
No. Observations:        82                  Deviance:          73.495   
Df Model:                5                   Pearson chi2:      73.5     
Df Residuals:            76                  Scale:             0.96704  
Method:                  IRLS                                            
-------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|   [0.025   0.975]
-------------------------------------------------------------------------
Intercept                 1.4119   0.2876  4.9090 0.0000   0.8482  1.9756
hmm_dprime_prev           0.4868   0.1079  4.5124 0.0000  

In [17]:
# Arthur 1 - no prev behavior
formula = 'hmm_dprime ~ d_day_prev + replay_plus_prev + replay_minus_prev + replay_neutral_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a1_no_ar = glm_results.aic

                   Results: Generalized linear model
Model:                  GLM                AIC:              257.0964 
Link Function:          identity           BIC:              -241.5869
Dependent Variable:     hmm_dprime         Log-Likelihood:   -123.55  
Date:                   2018-09-06 13:46   LL-Null:          -127.57  
No. Observations:       82                 Deviance:         97.730   
Df Model:               4                  Pearson chi2:     97.7     
Df Residuals:           77                 Scale:            1.2692   
Method:                 IRLS                                          
----------------------------------------------------------------------
                     Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
----------------------------------------------------------------------
Intercept             2.4950   0.1872 13.3279 0.0000    2.1281  2.8619
d_day_prev           -0.0132   0.0321 -0.4102 0.6817   -0.0760  0.0497
replay_plus_prev    -12.

In [18]:
# Arthur 2 - no previous behavior
formula = 'hmm_dprime ~ d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a2_no_ar = glm_results.aic

                     Results: Generalized linear model
Model:                  GLM                  AIC:                253.1916 
Link Function:          identity             BIC:                -246.1317
Dependent Variable:     hmm_dprime           Log-Likelihood:     -121.60  
Date:                   2018-09-06 13:46     LL-Null:            -127.57  
No. Observations:       82                   Deviance:           93.186   
Df Model:               4                    Pearson chi2:       93.2     
Df Residuals:           77                   Scale:              1.2102   
Method:                 IRLS                                              
--------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
--------------------------------------------------------------------------
Intercept                 2.4905   0.1789 13.9190 0.0000    2.1398  2.8412
d_day_prev               -0.0083   0.0315 -0.

In [19]:
aics = [aic_full, aic_null, aic_ar, aic_a1, aic_a2, aic_a1_no_ar, aic_a2_no_ar]
print("AICs: " + str(aics))
aic_min = min(aics)
delta_aic = [aic - aic_min for aic in aics]
print("Delta aics: " + str(delta_aic))
likelihoods = [np.exp(-1/2.* (x - aic_min)) for x in aics]
print("Likelihoods: " + str(likelihoods))
weights = [x / np.sum(likelihoods) for x in likelihoods]
print("Weights: " + str(weights))
weight_of_aic_min = max(weights)
evidence_ratios = [weight_of_aic_min / x for x in weights]
print("Evidence ratios: " + str(evidence_ratios))
# print("evidence ratio of a2/a1: " + str(0.35847911001635896/0.05302124507387048))
print("evidence ratio of a2/a1: {}".format(weights[4]/weights[3]))

AICs: [241.5002073984773, 257.13591062934535, 234.83535690215047, 239.58133224133832, 235.72661504611835, 257.09643862647033, 253.1916151415702]
Delta aics: [6.664850496326835, 22.30055372719488, 0.0, 4.745975339187851, 0.8912581439678888, 22.261081724319865, 18.35625823941973]
Likelihoods: [0.035706403084736556, 1.4371307647237989e-05, 1.0, 0.09320185355176067, 0.6404212782042469, 1.4657757181990642e-05, 0.00010327356500090968]
Weights: [0.020179244518648935, 8.121852273333915e-06, 0.5651435814120122, 0.052672429310479885, 0.36192997477680666, 8.283737389297835e-06, 5.8364392389800325e-05]
Evidence ratios: [28.00618134587381, 69583.09045678173, 1.0, 10.729400348724171, 1.5614721653284516, 68223.26141605464, 9683.020044782917]
evidence ratio of a2/a1: 6.8713362857


## Model Replay as function of change in behavior

In [20]:
# data_sub = data[[
#     'hmm_dprime_prev', 'hmm_dprime', 'hmm_dprime_next', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
#     'replay_freq_0_1_neutral', 'd_day_prev', 'd_day_next', 'replay_plus_prev', 'replay_minus_prev',
#     'replay_neutral_prev', 'replay_plus_next', 'replay_minus_next', 'replay_neutral_next']]
data_sub = data[[
    'd_hmm_dprime_prev', 'd_hmm_dprime_next', 'replay_freq_0_1_all', 'replay_freq_0_1_all_prev','d_day_prev', 'd_day_next']]
# data_sub = data[[
#     'd_hmm_dprime_prev', 'replay_freq_0_1_plus', 'replay_freq_0_1_minus',
#     'replay_freq_0_1_neutral', 'd_day_prev', 'replay_plus_prev', 'replay_minus_prev',
#     'replay_neutral_prev']]
data_sub.replace(0, np.nan, inplace=True)
print(data_sub.shape)
data_sub.dropna(inplace=True)
print(data_sub.shape)

(118, 6)
(63, 6)


In [21]:
# Full
formula = 'replay_freq_0_1_all ~ replay_freq_0_1_all_prev + d_day_prev + d_day_next' \
    + '+ d_hmm_dprime_prev + d_hmm_dprime_next'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_full = glm_results.aic

                    Results: Generalized linear model
Model:                  GLM                   AIC:              -449.9110
Link Function:          log                   BIC:              -163.4199
Dependent Variable:     replay_freq_0_1_all   Log-Likelihood:   230.96   
Date:                   2018-09-06 13:47      LL-Null:          224.57   
No. Observations:       63                    Deviance:         72.739   
Df Model:               5                     Pearson chi2:     66.4     
Df Residuals:           57                    Scale:            1.1643   
Method:                 IRLS                                             
-------------------------------------------------------------------------
                          Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-------------------------------------------------------------------------
Intercept                -5.1824   0.2225 -23.2951 0.0000 -5.6184 -4.7463
replay_freq_0_1_all_prev 49.0790  10.9550   4.4800 0.0000 

In [22]:
# Null
formula = 'replay_freq_0_1_all ~ 1'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_null = glm_results.aic

                Results: Generalized linear model
Model:              GLM                 AIC:            -435.3896
Link Function:      log                 BIC:            -169.2703
Dependent Variable: replay_freq_0_1_all Log-Likelihood: 218.69   
Date:               2018-09-06 13:47    LL-Null:        218.69   
No. Observations:   63                  Deviance:       87.604   
Df Model:           0                   Pearson chi2:   129.     
Df Residuals:       62                  Scale:          2.0859   
Method:             IRLS                                         
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept    -4.5551    0.1820  -25.0335  0.0000  -4.9118  -4.1985



In [23]:
# Autoregress
formula = 'replay_freq_0_1_all ~ replay_freq_0_1_all_prev + d_day_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_auto = glm_results.aic

                    Results: Generalized linear model
Model:                  GLM                   AIC:              -451.7049
Link Function:          log                   BIC:              -173.5185
Dependent Variable:     replay_freq_0_1_all   Log-Likelihood:   228.85   
Date:                   2018-09-06 13:47      LL-Null:          224.21   
No. Observations:       63                    Deviance:         75.070   
Df Model:               2                     Pearson chi2:     81.0     
Df Residuals:           60                    Scale:            1.3501   
Method:                 IRLS                                             
-------------------------------------------------------------------------
                          Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-------------------------------------------------------------------------
Intercept                -5.1842   0.2138 -24.2472 0.0000 -5.6033 -4.7651
replay_freq_0_1_all_prev 45.1373  11.6808   3.8642 0.0001 

In [24]:
# Replay changes behavior
formula = 'replay_freq_0_1_all ~ replay_freq_0_1_all_prev + d_day_prev + d_day_next + d_hmm_dprime_next'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_fwd = glm_results.aic

                    Results: Generalized linear model
Model:                  GLM                   AIC:              -447.1809
Link Function:          log                   BIC:              -165.3481
Dependent Variable:     replay_freq_0_1_all   Log-Likelihood:   228.59   
Date:                   2018-09-06 13:47      LL-Null:          224.04   
No. Observations:       63                    Deviance:         74.954   
Df Model:               4                     Pearson chi2:     80.6     
Df Residuals:           58                    Scale:            1.3899   
Method:                 IRLS                                             
-------------------------------------------------------------------------
                          Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-------------------------------------------------------------------------
Intercept                -5.1651   0.2422 -21.3289 0.0000 -5.6398 -4.6905
replay_freq_0_1_all_prev 45.2193  11.8730   3.8086 0.0001 

In [25]:
# Replay reflects behavior change
formula = 'replay_freq_0_1_all ~ replay_freq_0_1_all_prev + d_day_prev + d_day_prev + d_hmm_dprime_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_bkw = glm_results.aic

                    Results: Generalized linear model
Model:                  GLM                   AIC:              -453.3266
Link Function:          log                   BIC:              -171.0779
Dependent Variable:     replay_freq_0_1_all   Log-Likelihood:   230.66   
Date:                   2018-09-06 13:47      LL-Null:          224.57   
No. Observations:       63                    Deviance:         73.367   
Df Model:               3                     Pearson chi2:     69.0     
Df Residuals:           59                    Scale:            1.1690   
Method:                 IRLS                                             
-------------------------------------------------------------------------
                          Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-------------------------------------------------------------------------
Intercept                -5.1870   0.1999 -25.9496 0.0000 -5.5787 -4.7952
replay_freq_0_1_all_prev 48.0834  10.9624   4.3862 0.0000 

In [26]:
# Replay changes behavior - no auto
formula = 'replay_freq_0_1_all ~ d_day_next + d_hmm_dprime_next'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_fwd_no_auto = glm_results.aic

                 Results: Generalized linear model
Model:               GLM                 AIC:            -430.1793
Link Function:       log                 BIC:            -161.0516
Dependent Variable:  replay_freq_0_1_all Log-Likelihood: 218.09   
Date:                2018-09-06 13:47    LL-Null:        218.07   
No. Observations:    63                  Deviance:       87.536   
Df Model:            2                   Pearson chi2:   129.     
Df Residuals:        60                  Scale:          2.1527   
Method:              IRLS                                         
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
Intercept         -4.5308   0.2283 -19.8499 0.0000 -4.9781 -4.0834
d_day_next        -0.0073   0.0417  -0.1752 0.8610 -0.0891  0.0745
d_hmm_dprime_next -0.0241   0.1631  -0.1479 0.8824 -0.3439  0.2956



In [27]:
# Replay reflects beahvior change - no auto
formula = 'replay_freq_0_1_all ~ d_day_prev + d_hmm_dprime_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_bkw_no_auto = glm_results.aic

                 Results: Generalized linear model
Model:               GLM                 AIC:            -433.0704
Link Function:       log                 BIC:            -161.8347
Dependent Variable:  replay_freq_0_1_all Log-Likelihood: 219.54   
Date:                2018-09-06 13:47    LL-Null:        219.32   
No. Observations:    63                  Deviance:       86.753   
Df Model:            2                   Pearson chi2:   121.     
Df Residuals:        60                  Scale:          2.0173   
Method:              IRLS                                         
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
Intercept         -4.5943   0.2191 -20.9733 0.0000 -5.0236 -4.1649
d_day_prev         0.0197   0.0418   0.4708 0.6378 -0.0623  0.1017
d_hmm_dprime_prev -0.1014   0.1891  -0.5362 0.5918 -0.4721  0.2693



In [28]:
aics = [aic_full, aic_null, aic_auto, aic_fwd, aic_bkw, aic_fwd_no_auto, aic_bkw_no_auto]
print("AICs: " + str(aics))
aic_min = min(aics)
delta_aic = [aic - aic_min for aic in aics]
print("Delta aics: " + str(delta_aic))
likelihoods = [np.exp(-1/2.* (x - aic_min)) for x in aics]
print("Likelihoods: " + str(likelihoods))
weights = [x / np.sum(likelihoods) for x in likelihoods]
print("Weights: " + str(weights))
weight_of_aic_min = max(weights)
evidence_ratios = [weight_of_aic_min / x for x in weights]
print("Evidence ratios: " + str(evidence_ratios))
print("evidence ratio of bkw/fwd: {}".format(weights[4]/weights[3]))

AICs: [-449.9109548150844, -435.38959076106715, -451.70488264730824, -447.18086989069144, -453.3266347945212, -430.17925412762133, -433.07036583201625]
Delta aics: [3.4156799794368453, 17.93704403345407, 1.6217521472129874, 6.145764903829786, 0.0, 23.147380666899892, 20.256268962504976]
Likelihoods: [0.18125688682045196, 0.00012735628358004326, 0.44446850847803226, 0.04628754077132147, 1.0, 9.410444829843021e-06, 3.993991008414245e-05]
Weights: [0.10839493451644994, 7.616138763649761e-05, 0.2658002998739816, 0.027680796238130967, 0.5980182955686696, 5.627618177685723e-06, 2.388479695368479e-05]
Evidence ratios: [5.5170317527883626, 7851.987918377826, 2.2498781824256615, 21.604085750426677, 1.0, 106264.90225294497, 25037.612701011938]
evidence ratio of bkw/fwd: 21.6040857504


# With all replay combined

## Modeling behavior

In [29]:
data_sub = data[[
    'hmm_dprime_prev', 'hmm_dprime', 'd_day_prev',
#     'replay_freq_0_1_plus', 'replay_freq_0_1_minus', 'replay_freq_0_1_neutral',
#     'replay_plus_prev', 'replay_minus_prev', 'replay_neutral_prev',
    'combrep_freq_0_1_plus', 'combrep_freq_0_1_minus', 'combrep_freq_0_1_neutral',
    'replay_combined_plus_prev', 'replay_combined_minus_prev', 'replay_combined_neutral_prev',
#     'replay_freq_hungry_0_1_plus', 'replay_freq_hungry_0_1_minus', 'replay_freq_hungry_0_1_neutral',
#     'replay_hungry_plus_prev', 'replay_hungry_minus_prev', 'replay_hungry_neutral_prev',
#     'replay_freq_iti_0_1_plus', 'replay_freq_iti_0_1_minus', 'replay_freq_iti_0_1_neutral',
#     'replay_iti_plus_prev', 'replay_iti_minus_prev', 'replay_iti_neutral_prev',
]]
print(data_sub.shape)
data_sub.dropna(inplace=True)
print(data_sub.shape)

(118, 9)
(82, 9)


In [30]:
# Full
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev ' \
    + '+ combrep_freq_0_1_plus + combrep_freq_0_1_minus + combrep_freq_0_1_neutral ' \
    + '+ replay_combined_plus_prev + replay_combined_minus_prev + replay_combined_neutral_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_full = glm_results.aic

                        Results: Generalized linear model
Model:                    GLM                    AIC:                  241.5096 
Link Function:            identity               BIC:                  -248.3899
Dependent Variable:       hmm_dprime             Log-Likelihood:       -111.75  
Date:                     2018-09-06 13:47       LL-Null:              -127.57  
No. Observations:         82                     Deviance:             73.301   
Df Model:                 8                      Pearson chi2:         73.3     
Df Residuals:             73                     Scale:                1.0041   
Method:                   IRLS                                                  
--------------------------------------------------------------------------------
                              Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
--------------------------------------------------------------------------------
Intercept                      1.3454   0.3319  4.0

In [31]:
# Null
formula = 'hmm_dprime ~ 1'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_null = glm_results.aic

               Results: Generalized linear model
Model:              GLM              AIC:            257.1359 
Link Function:      identity         BIC:            -249.1466
Dependent Variable: hmm_dprime       Log-Likelihood: -127.57  
Date:               2018-09-06 13:47 LL-Null:        -127.57  
No. Observations:   82               Deviance:       107.80   
Df Model:           0                Pearson chi2:   108.     
Df Residuals:       81               Scale:          1.3308   
Method:             IRLS                                      
---------------------------------------------------------------
              Coef.   Std.Err.     z     P>|z|   [0.025  0.975]
---------------------------------------------------------------
Intercept     2.2238    0.1274  17.4561  0.0000  1.9741  2.4735



In [32]:
# Autoregress
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_ar = glm_results.aic

               Results: Generalized linear model
Model:              GLM              AIC:            234.8354 
Link Function:      identity         BIC:            -269.9115
Dependent Variable: hmm_dprime       Log-Likelihood: -114.42  
Date:               2018-09-06 13:47 LL-Null:        -127.57  
No. Observations:   82               Deviance:       78.219   
Df Model:           2                Pearson chi2:   78.2     
Df Residuals:       79               Scale:          0.99012  
Method:             IRLS                                      
--------------------------------------------------------------
                 Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
--------------------------------------------------------------
Intercept        1.1362   0.2462  4.6148 0.0000  0.6536 1.6187
hmm_dprime_prev  0.5477   0.1005  5.4509 0.0000  0.3508 0.7446
d_day_prev      -0.0155   0.0283 -0.5493 0.5828 -0.0710 0.0399



In [33]:
# Arthur 1
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev + replay_combined_plus_prev + replay_combined_minus_prev + replay_combined_neutral_prev'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_plus_prev + replay_minus_prev + replay_neutral_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a1 = glm_results.aic

                        Results: Generalized linear model
Model:                    GLM                    AIC:                  240.6503 
Link Function:            identity               BIC:                  -256.8676
Dependent Variable:       hmm_dprime             Log-Likelihood:       -114.33  
Date:                     2018-09-06 13:47       LL-Null:              -127.57  
No. Observations:         82                     Deviance:             78.043   
Df Model:                 5                      Pearson chi2:         78.0     
Df Residuals:             76                     Scale:                1.0269   
Method:                   IRLS                                                  
--------------------------------------------------------------------------------
                              Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
--------------------------------------------------------------------------------
Intercept                      1.1656   0.3147  3.7

In [34]:
# Arthur 2
formula = 'hmm_dprime ~ hmm_dprime_prev + d_day_prev + combrep_freq_0_1_plus + combrep_freq_0_1_minus + combrep_freq_0_1_neutral'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a2 = glm_results.aic

                     Results: Generalized linear model
Model:                   GLM                  AIC:                236.0535 
Link Function:           identity             BIC:                -261.1222
Dependent Variable:      hmm_dprime           Log-Likelihood:     -112.03  
Date:                    2018-09-06 13:47     LL-Null:            -127.57  
No. Observations:        82                   Deviance:           73.788   
Df Model:                5                    Pearson chi2:       73.8     
Df Residuals:            76                   Scale:              0.97090  
Method:                  IRLS                                              
---------------------------------------------------------------------------
                          Coef.   Std.Err.    z    P>|z|    [0.025   0.975]
---------------------------------------------------------------------------
Intercept                  1.3945   0.2865  4.8671 0.0000    0.8329  1.9560
hmm_dprime_prev            0.5017

In [35]:
# Arthur 1 - no prev behavior
formula = 'hmm_dprime ~ d_day_prev + replay_combined_plus_prev + replay_combined_minus_prev + replay_combined_neutral_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a1_no_ar = glm_results.aic

                        Results: Generalized linear model
Model:                    GLM                    AIC:                  260.1389 
Link Function:            identity               BIC:                  -237.8927
Dependent Variable:       hmm_dprime             Log-Likelihood:       -125.07  
Date:                     2018-09-06 13:47       LL-Null:              -127.57  
No. Observations:         82                     Deviance:             101.42   
Df Model:                 4                      Pearson chi2:         101.     
Df Residuals:             77                     Scale:                1.3172   
Method:                   IRLS                                                  
--------------------------------------------------------------------------------
                              Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
--------------------------------------------------------------------------------
Intercept                      2.3949   0.2047 11.6

In [36]:
# Arthur 2 - no previous behavior
formula = 'hmm_dprime ~ d_day_prev + combrep_freq_0_1_plus + combrep_freq_0_1_minus + combrep_freq_0_1_neutral'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a2_no_ar = glm_results.aic

                      Results: Generalized linear model
Model:                    GLM                  AIC:                254.8918 
Link Function:            identity             BIC:                -244.1795
Dependent Variable:       hmm_dprime           Log-Likelihood:     -122.45  
Date:                     2018-09-06 13:47     LL-Null:            -127.57  
No. Observations:         82                   Deviance:           95.138   
Df Model:                 4                    Pearson chi2:       95.1     
Df Residuals:             77                   Scale:              1.2356   
Method:                   IRLS                                              
----------------------------------------------------------------------------
                          Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
----------------------------------------------------------------------------
Intercept                  2.4741   0.1924 12.8595 0.0000    2.0970   2.8511
d_day_prev          

In [37]:
aics = [aic_full, aic_null, aic_ar, aic_a1, aic_a2, aic_a1_no_ar, aic_a2_no_ar]
print("AICs: " + str(aics))
aic_min = min(aics)
delta_aic = [aic - aic_min for aic in aics]
print("Delta aics: " + str(delta_aic))
likelihoods = [np.exp(-1/2.* (x - aic_min)) for x in aics]
print("Likelihoods: " + str(likelihoods))
weights = [x / np.sum(likelihoods) for x in likelihoods]
print("Weights: " + str(weights))
weight_of_aic_min = max(weights)
evidence_ratios = [weight_of_aic_min / x for x in weights]
print("Evidence ratios: " + str(evidence_ratios))
print("evidence ratio of a2/a1: {}".format(weights[4]/weights[3]))

AICs: [241.50962932544036, 257.13591062934535, 234.83535690215047, 240.65031974748615, 236.05350614894138, 260.138904603535, 254.8918026821865]
Delta aics: [6.674272423289892, 22.30055372719488, 0.0, 5.814962845335685, 1.2181492467909152, 25.30354770138456, 20.05644578003603]
Likelihoods: [0.03553858712154872, 1.4371307647237989e-05, 1.0, 0.054613104128822906, 0.5438539059701896, 3.2018754212408313e-06, 4.413652486737933e-05]
Weights: [0.021748545467413723, 8.794807647336917e-06, 0.6119698960746405, 0.03342157565802927, 0.33282221831636416, 1.959451368780697e-06, 2.701022453618591e-05]
Evidence ratios: [28.138428705108904, 69583.09045678173, 1.0, 18.310623722123033, 1.8387290943807497, 312316.9606681535, 22656.971816534784]
evidence ratio of a2/a1: 9.95830423203


## Modeling reactivation

In [38]:
data_sub = data[[
    'd_hmm_dprime_prev', 'd_hmm_dprime_next', 'd_day_prev', 'd_day_next',
    'combrep_freq_0_1_all', 'combrep_freq_0_1_all_prev', 'mouse']]
data_sub.replace(0, np.nan, inplace=True)
print(data_sub.shape)
data_sub.dropna(inplace=True)
data_sub['d_day_prev_cent'] = data_sub['d_day_prev'] - data_sub['d_day_prev'].mean()
data_sub['combrep_freq_0_1_all_prev_cent'] = data_sub['combrep_freq_0_1_all_prev'] - data_sub['combrep_freq_0_1_all_prev'].mean()
print(data_sub.shape)

(118, 7)
(67, 9)


In [39]:
# Full
formula = 'combrep_freq_0_1_all ~ combrep_freq_0_1_all_prev + d_day_prev ' \
    + '+ d_hmm_dprime_next + d_day_next' \
    + '+ d_hmm_dprime_prev + d_day_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_full = glm_results.aic

                     Results: Generalized linear model
Model:                  GLM                    AIC:              -515.2987
Link Function:          log                    BIC:              -193.0808
Dependent Variable:     combrep_freq_0_1_all   Log-Likelihood:   263.65   
Date:                   2018-09-06 13:47       LL-Null:          259.20   
No. Observations:       67                     Deviance:         63.405   
Df Model:               5                      Pearson chi2:     61.8     
Df Residuals:           61                     Scale:            1.0127   
Method:                 IRLS                                              
--------------------------------------------------------------------------
                           Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------------
Intercept                 -5.2549   0.2032 -25.8648 0.0000 -5.6531 -4.8567
combrep_freq_0_1_all_prev 43.3437  13.8568   

In [40]:
# Null
formula = 'combrep_freq_0_1_all ~ 1'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_null = glm_results.aic

                 Results: Generalized linear model
Model:              GLM                  AIC:            -509.0979
Link Function:      log                  BIC:            -205.0864
Dependent Variable: combrep_freq_0_1_all Log-Likelihood: 255.55   
Date:               2018-09-06 13:47     LL-Null:        255.55   
No. Observations:   67                   Deviance:       72.423   
Df Model:           0                    Pearson chi2:   97.0     
Df Residuals:       66                   Scale:          1.4694   
Method:             IRLS                                          
-------------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
Intercept     -4.8691    0.1481  -32.8785  0.0000  -5.1594  -4.5789



In [41]:
# Autoregress
formula = 'combrep_freq_0_1_all ~ combrep_freq_0_1_all_prev + d_day_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_auto = glm_results.aic

                     Results: Generalized linear model
Model:                  GLM                    AIC:              -514.6220
Link Function:          log                    BIC:              -202.1170
Dependent Variable:     combrep_freq_0_1_all   Log-Likelihood:   260.31   
Date:                   2018-09-06 13:47       LL-Null:          258.06   
No. Observations:       67                     Deviance:         66.983   
Df Model:               2                      Pearson chi2:     77.4     
Df Residuals:           64                     Scale:            1.2094   
Method:                 IRLS                                              
--------------------------------------------------------------------------
                           Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------------
Intercept                 -5.2459   0.1998 -26.2621 0.0000 -5.6375 -4.8544
combrep_freq_0_1_all_prev 36.2488  14.9116   

In [42]:
# Replay changes behavior - no auto
formula = 'combrep_freq_0_1_all ~ d_day_next + d_hmm_dprime_next'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_fwd_no_auto = glm_results.aic

                 Results: Generalized linear model
Model:              GLM                  AIC:            -505.8456
Link Function:      log                  BIC:            -197.1667
Dependent Variable: combrep_freq_0_1_all Log-Likelihood: 255.92   
Date:               2018-09-06 13:47     LL-Null:        255.75   
No. Observations:   67                   Deviance:       71.934   
Df Model:           2                    Pearson chi2:   92.8     
Df Residuals:       64                   Scale:          1.4505   
Method:             IRLS                                          
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
Intercept         -4.8666   0.1823 -26.6964 0.0000 -5.2239 -4.5093
d_day_next        -0.0044   0.0342  -0.1287 0.8976 -0.0714  0.0626
d_hmm_dprime_next  0.0874   0.1334   0.6552 0.5123 -0.1741  0.3489



In [43]:
# Replay reflects behavior change - no auto
formula = 'combrep_freq_0_1_all ~ d_day_prev + d_hmm_dprime_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_bkw_no_auto = glm_results.aic

                 Results: Generalized linear model
Model:              GLM                  AIC:            -508.2764
Link Function:      log                  BIC:            -198.8508
Dependent Variable: combrep_freq_0_1_all Log-Likelihood: 257.14   
Date:               2018-09-06 13:47     LL-Null:        256.36   
No. Observations:   67                   Deviance:       70.249   
Df Model:           2                    Pearson chi2:   89.2     
Df Residuals:       64                   Scale:          1.3932   
Method:             IRLS                                          
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
Intercept         -4.8923   0.1777 -27.5383 0.0000 -5.2405 -4.5441
d_day_prev         0.0178   0.0347   0.5149 0.6066 -0.0501  0.0858
d_hmm_dprime_prev -0.1786   0.1550  -1.1523 0.2492 -0.4824  0.1252



In [44]:
# Replay changes behavior
formula = 'combrep_freq_0_1_all ~ combrep_freq_0_1_all_prev + d_day_prev + d_day_next + d_hmm_dprime_next'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_fwd = glm_results.aic

                     Results: Generalized linear model
Model:                  GLM                    AIC:              -512.0205
Link Function:          log                    BIC:              -194.4154
Dependent Variable:     combrep_freq_0_1_all   Log-Likelihood:   261.01   
Date:                   2018-09-06 13:47       LL-Null:          258.38   
No. Observations:       67                     Deviance:         66.276   
Df Model:               4                      Pearson chi2:     72.4     
Df Residuals:           62                     Scale:            1.1679   
Method:                 IRLS                                              
--------------------------------------------------------------------------
                           Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------------
Intercept                 -5.2346   0.2168 -24.1415 0.0000 -5.6596 -4.8096
combrep_freq_0_1_all_prev 36.7452  14.7882   

In [45]:
# Replay reflects behavior change
formula = 'combrep_freq_0_1_all ~ combrep_freq_0_1_all_prev + d_day_prev + d_day_prev + d_hmm_dprime_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gamma(link=sm.families.links.log)

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_bkw = glm_results.aic

                     Results: Generalized linear model
Model:                  GLM                    AIC:              -519.7002
Link Function:          log                    BIC:              -201.4788
Dependent Variable:     combrep_freq_0_1_all   Log-Likelihood:   263.85   
Date:                   2018-09-06 13:47       LL-Null:          259.27   
No. Observations:       67                     Deviance:         63.417   
Df Model:               3                      Pearson chi2:     61.9     
Df Residuals:           63                     Scale:            0.98271  
Method:                 IRLS                                              
--------------------------------------------------------------------------
                           Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------------
Intercept                 -5.2642   0.1809 -29.0929 0.0000 -5.6189 -4.9096
combrep_freq_0_1_all_prev 43.3855  13.5764   

In [46]:
aics = [aic_full, aic_null, aic_auto, aic_fwd_no_auto, aic_bkw_no_auto, aic_fwd, aic_bkw]
print("AICs: " + str(aics))
aic_min = min(aics)
delta_aic = [aic - aic_min for aic in aics]
print("Delta aics: " + str(delta_aic))
likelihoods = [np.exp(-1/2.* (x - aic_min)) for x in aics]
print("Likelihoods: " + str(likelihoods))
weights = [x / np.sum(likelihoods) for x in likelihoods]
print("Weights: " + str(weights))
weight_of_aic_min = max(weights)
evidence_ratios = [weight_of_aic_min / x for x in weights]
print("Evidence ratios: " + str(evidence_ratios))
print("evidence ratio of bkw/fwd: {}".format(weights[6]/weights[5]))

AICs: [-515.298701457322, -509.09785967965394, -514.622032039474, -505.84563985931317, -508.2764080723513, -512.020497509024, -519.700243489692]
Delta aics: [4.401542032370003, 10.60238381003802, 5.078211450217964, 13.85460363037879, 11.423835417340683, 7.679745980667917, 0.0]
Likelihoods: [0.11071776025977739, 0.004985647945294377, 0.07893695957697296, 0.0009806432587088666, 0.003306325891499703, 0.02149633141358766, 1.0]
Weights: [0.0907207579887761, 0.004085178020229574, 0.06467996452737099, 0.0008035269096657458, 0.002709162381274602, 0.017613827043131452, 0.8193875831295515]
Evidence ratios: [9.031974614133246, 200.57573478364708, 12.668336928088552, 1019.7388205335942, 302.4505244842679, 46.5195656300641, 1.0]
evidence ratio of bkw/fwd: 46.5195656301


## Model d behavior

In [47]:
data_sub = data[[
    'hmm_dprime_prev', 'hmm_dprime', 'd_day_prev',
#     'replay_freq_0_1_plus', 'replay_freq_0_1_minus', 'replay_freq_0_1_neutral',
#     'replay_plus_prev', 'replay_minus_prev', 'replay_neutral_prev',
    'combrep_freq_0_1_plus', 'combrep_freq_0_1_minus', 'combrep_freq_0_1_neutral',
    'replay_combined_plus_prev', 'replay_combined_minus_prev', 'replay_combined_neutral_prev',
#     'replay_freq_hungry_0_1_plus', 'replay_freq_hungry_0_1_minus', 'replay_freq_hungry_0_1_neutral',
#     'replay_hungry_plus_prev', 'replay_hungry_minus_prev', 'replay_hungry_neutral_prev',
#     'replay_freq_iti_0_1_plus', 'replay_freq_iti_0_1_minus', 'replay_freq_iti_0_1_neutral',
#     'replay_iti_plus_prev', 'replay_iti_minus_prev', 'replay_iti_neutral_prev',
]]
print(data_sub.shape)
data_sub.replace(0, np.nan, inplace=True)
data_sub.dropna(inplace=True)
data_sub['d_hmm_dprime'] = data_sub['hmm_dprime'] - data_sub['hmm_dprime_prev']
print(data_sub.shape)

(118, 9)
(61, 10)


In [48]:
# Full
formula = 'd_hmm_dprime ~ d_day_prev ' \
    + '+ combrep_freq_0_1_plus + combrep_freq_0_1_minus + combrep_freq_0_1_neutral ' \
    + '+ replay_combined_plus_prev + replay_combined_minus_prev + replay_combined_neutral_prev'

y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_full = glm_results.aic

                        Results: Generalized linear model
Model:                     GLM                    AIC:                  192.4416 
Link Function:             identity               BIC:                  -153.4526
Dependent Variable:        d_hmm_dprime           Log-Likelihood:       -88.221  
Date:                      2018-09-06 13:47       LL-Null:              -93.088  
No. Observations:          61                     Deviance:             64.424   
Df Model:                  7                      Pearson chi2:         64.4     
Df Residuals:              53                     Scale:                1.2155   
Method:                    IRLS                                                  
---------------------------------------------------------------------------------
                               Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
---------------------------------------------------------------------------------
Intercept                       0.5165  

In [49]:
# Null
formula = 'd_hmm_dprime ~ 1'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_null = glm_results.aic

               Results: Generalized linear model
Model:              GLM              AIC:            188.1753 
Link Function:      identity         BIC:            -171.0831
Dependent Variable: d_hmm_dprime     Log-Likelihood: -93.088  
Date:               2018-09-06 13:47 LL-Null:        -93.088  
No. Observations:   61               Deviance:       75.569   
Df Model:           0                Pearson chi2:   75.6     
Df Residuals:       60               Scale:          1.2595   
Method:             IRLS                                      
---------------------------------------------------------------
              Coef.   Std.Err.    z     P>|z|    [0.025  0.975]
---------------------------------------------------------------
Intercept     0.2404    0.1437  1.6733  0.0943  -0.0412  0.5221



In [50]:
# Arthur 1
formula = 'd_hmm_dprime ~d_day_prev + replay_combined_plus_prev + replay_combined_minus_prev + replay_combined_neutral_prev'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_plus_prev + replay_minus_prev + replay_neutral_prev'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a1 = glm_results.aic

                       Results: Generalized linear model
Model:                     GLM                   AIC:                 194.2666 
Link Function:             identity              BIC:                 -156.9675
Dependent Variable:        d_hmm_dprime          Log-Likelihood:      -92.133  
Date:                      2018-09-06 13:47      LL-Null:             -93.088  
No. Observations:          61                    Deviance:            73.241   
Df Model:                  4                     Pearson chi2:        73.2     
Df Residuals:              56                    Scale:               1.3079   
Method:                    IRLS                                                
-------------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|    [0.025   0.975] 
-------------------------------------------------------------------------------
Intercept                     0.1896   0.2563  0.7397 0.4595   

In [51]:
# Arthur 2
formula = 'd_hmm_dprime ~ d_day_prev + combrep_freq_0_1_plus + combrep_freq_0_1_minus + combrep_freq_0_1_neutral'
# formula = 'd_hmm_dprime_prev ~ d_day_prev + replay_freq_0_1_plus + replay_freq_0_1_minus + replay_freq_0_1_neutral'
y, X = patsy.dmatrices(formula, data_sub, return_type='dataframe')
X = sm.add_constant(X)
family = sm.families.Gaussian()

glm = sm.GLM(y, X, family=family)
glm_results = glm.fit()
print(glm_results.summary2())
aic_a2 = glm_results.aic

                      Results: Generalized linear model
Model:                   GLM                   AIC:                 187.4005 
Link Function:           identity              BIC:                 -164.7645
Dependent Variable:      d_hmm_dprime          Log-Likelihood:      -88.700  
Date:                    2018-09-06 13:47      LL-Null:             -93.088  
No. Observations:        61                    Deviance:            65.444   
Df Model:                4                     Pearson chi2:        65.4     
Df Residuals:            56                    Scale:               1.1687   
Method:                  IRLS                                                
-----------------------------------------------------------------------------
                           Coef.   Std.Err.    z    P>|z|    [0.025   0.975] 
-----------------------------------------------------------------------------
Intercept                   0.6625   0.2303  2.8761 0.0040    0.2110   1.1139
d_day_pr

In [52]:
aics = [aic_full, aic_null, aic_a1, aic_a2]
print("AICs: " + str(aics))
aic_min = min(aics)
delta_aic = [aic - aic_min for aic in aics]
print("Delta aics: " + str(delta_aic))
likelihoods = [np.exp(-1/2.* (x - aic_min)) for x in aics]
print("Likelihoods: " + str(likelihoods))
weights = [x / np.sum(likelihoods) for x in likelihoods]
print("Weights: " + str(weights))
weight_of_aic_min = max(weights)
evidence_ratios = [weight_of_aic_min / x for x in weights]
print("Evidence ratios: " + str(evidence_ratios))
print("evidence ratio of a2/a1: {}".format(weights[3]/weights[2]))

AICs: [192.4416175243739, 188.17528070764038, 194.26660953136783, 187.4004699509091]
Delta aics: [5.041147573464798, 0.7748107567312843, 6.866139580458736, 0.0]
Likelihoods: [0.08041345333705964, 0.6788158618789839, 0.03228767210351472, 1.0]
Weights: [0.044885677281449106, 0.3789056239397531, 0.018022531927996435, 0.5581861668508014]
Evidence ratios: [12.435730073777794, 1.473153554827031, 30.9715731996406, 1.0]
evidence ratio of a2/a1: 30.9715731996
